In [1]:
import pandas as pd
import numpy as np
import feather
#import statsmodels.formula.api as smf
#from statsmodels.iolib.summary2 import summary_col
#import seaborn as sns
#import matplotlib.pyplot as plt
import os
#import altair as alt
from pathlib import Path

## Import data 

In [2]:
datadir = "../input"
Path(datadir).mkdir(parents=True, exist_ok=True) # Create if doesn't exist

In [3]:
os.chdir(datadir)
patents_df = pd.read_feather("solar_dataset_all_renewables") 
patents_df.head()

,appln_id,appln_auth,appln_kind,appln_filing_date,appln_filing_year,receiving_office,earliest_filing_year,earliest_filing_id,earliest_publn_year,granted,...,nb_citing_docdb_fam,ipc_class_symbol,ipc_class_level,ipc_value,fossil_fuel,person_ctry_code,cpc_class_symbol,cpc_gener_auth,cpc_position,cpc_value
0,66,EP,A,2004-03-02,2004,,2003,905739597,2005,Y,...,14,F27B5/04,A,I,1.0,FR,None,None,None,None
1,103,EP,A,2006-04-13,2006,,2005,6502820,2007,N,...,4,F01K21/04,A,I,1.0,CH,None,None,None,None
2,127,EP,A,2006-10-02,2006,,2005,1109097,2008,Y,...,10,F27D3/02,A,I,1.0,AT,None,None,None,None
3,148,EP,A,2007-09-27,2007,,2006,15054845,2008,Y,...,11,F27B17/02,A,I,1.0,AT,None,None,None,None
4,256,EP,A,2008-06-06,2008,,2007,55458188,2009,Y,...,5,F23N1/02,A,I,1.0,DE,None,None,None,None


In [4]:
patents_df["country_code"] = patents_df['person_ctry_code']
# patents_df["country_code"] = patents_df["appln_auth"] # cover all observations but assign the authority country not the innovator country. This mostly concerns Chinese authority applications. 

In [5]:
# Classify patents as fossil fuel or renewables

patents_df.loc[patents_df["cpc_class_symbol"].str.startswith("Y02E", na=False), 'cpc_renewables'] = 1
patents_df.loc[~patents_df["cpc_class_symbol"].str.startswith("Y02E", na=False), 'cpc_renewables'] = 0

# Solar only:
patents_df.loc[patents_df["cpc_class_symbol"].str.startswith("Y02E10/5", na=False), 'cpc_solar'] = 1
patents_df.loc[~patents_df["cpc_class_symbol"].str.startswith("Y02E10/5", na=False), 'cpc_solar'] = 0

In [40]:
patents_df=patents_df.dropna(subset=["country_code"]).reset_index()

In [41]:
# List all countries patenting 
patents_df["country_code"].unique()
list_countries = patents_df["country_code"].unique().tolist()

## Count patents 

In [43]:
### Absolute count ###

def absolute_count(df, list_countries):

    ## fossil fuel ##
    patents_year = df[df.fossil_fuel == 1].groupby('earliest_filing_year')['docdb_family_id'].nunique().to_frame(name = "Nb_ff_energy_innov_per_year") # Transform the series that groupby creates to a dataframe

    # countries #
    dict = {} 
    for country in list_countries:
        name = "Nb_ff_energy_innov_per_year_" + country
        value = df[(df.fossil_fuel == 1) & (df.country_code == country)].groupby('earliest_filing_year')['docdb_family_id'].nunique() 
        dict["Nb_ff_energy_innov_per_year_" + country] = value
        patents_year[name] = dict[name] 


    ## Renewables panel ##
    patents_year["Nb_renewables_energy_innov_per_year"] = df[df["cpc_renewables"] == 1].groupby('earliest_filing_year')['docdb_family_id'].nunique()

    # countries #
    # by family
    dict = {} 
    for country in list_countries:
        name = "Nb_renewables_energy_innov_per_year_" + country
        value = df[(df["cpc_renewables"] == 1) & (df.country_code == country)].groupby('earliest_filing_year')['docdb_family_id'].nunique()
        dict["Nb_renewables_energy_innov_per_year_" + country] = value
        patents_year[name] = dict[name] 

    
    return patents_year

In [45]:
# Save to feather 
os.chdir(datadir)
absolute_count(patents_df, list_countries).reset_index().to_feather("patents_df")

/tmp/ipykernel_9612/1853555206.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  patents_year[name] = dict[name]
/tmp/ipykernel_9612/1853555206.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  patents_year[name] = dict[name]
/tmp/ipykernel_9612/1853555206.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f